In [1]:
# !pip install poe-api
!pip install -U poe-api-wrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=2eb67cc78d223c14dc3ba69c8dca03a98390f7759a6d1a1a036bd3bc237a7f6c
  Stored in dir

In [2]:
from poe_api_wrapper import PoeApi
client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")

# client = PoeApi("p49wzQYujpnuLVWJ8eOKDw%3D%3D")

# Using Client with proxy (default is False)
# client = PoeApi("TOKEN_HERE", proxy=True)

**Mounted drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**get data**

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/drive/MyDrive/largeProject/code/LLM training/Splitted Visquad /df_gen_6000.csv",index_col=False)

In [6]:
def get_prompt(context, question):
    prompt = f'''Ngữ cảnh: "{context}". Câu hỏi "{question}". Trả lời câu hỏi theo từng bước dựa vào thông tin trong ngữ cảnh trên với tối đa 100 từ.'''
    return prompt.strip()

In [7]:
# print(client.get_chat_history('chinchilla'))
dict_client = client.get_chat_history()

In [8]:
list_chat_code = []

for dict_id in dict_client['acouchy']:
  list_chat_code.append(dict_id['chatCode'])

In [10]:
# client.delete_chat('acouchy', del_all=True)

Chat 81656902 deleted
Chat 81661015 deleted
Chat 81660162 deleted
Chat 81659638 deleted
Chat 81660710 deleted
Chat 81657844 deleted
Chat 81657484 deleted


In [9]:
list_chat_code

['2jbq7rsbntxbi0jtyk8',
 '2jbpvoezqyn0vrvlry3',
 '2jbq0f1z6c9wn1aypln',
 '2jbq0xo19cwzlpmhejh',
 '2jbqejodrdlcdlx3lfy',
 '2jbq5dowt6c3lhxbw36',
 '2jbq5kwyt7i8411z58j']

In [10]:
df['prompt'] = df.apply(lambda x: get_prompt(x['context'], x['question']), axis=1)

In [ ]:
df['prompt'].to_list()[:10]

In [13]:
import os
import random
from tqdm import tqdm
import time

In [ ]:
list_res = []

for index, row in tqdm(df.iterrows()):
    if index <= 800:
      continue
    prompt = row['prompt']
    res = ""
    while True:
        try:
            for chunk in client.send_message("acouchy", prompt, chatCode = list_chat_code[index % len(list_chat_code)], timeout=30):
                pass
            res = chunk["text"]
            if res == "You are sending and receiving too many words in a short period of time.":
                continue
            break
        except Exception as e:
            print(e)
            if "Timed out waiting" in str(e):
                client = PoeApi("3YCojFePwaecuxQp1n391g%3D%3D")
                dict_chat = client.get_chat_history()["acouchy"]
                list_chat_code = [each['chatCode'] for each in dict_chat]
                time.sleep(10)
            res = ""
    res = " ".join(res.split("\n")).strip()

    list_res.append((index, res))
    if index % 10 == 0:
        time.sleep(10)
    if index % 100 == 0:
        clone_ = df.loc[801:index].copy()
        clone_ = clone_.drop(columns = ['prompt'])
        clone_['answer'] = list_res
        clone_.to_csv(f'/content/drive/MyDrive/largeProject/processData/mrc_6000_{index}.csv', index=False)

802it [00:21, 79.85it/s]


Retrying request 1/3 times...


804it [01:06,  8.94it/s]


Retrying request 1/3 times...


805it [01:43,  4.83it/s]

...


Retrying request 1/3 times...


914it [1:40:53, 49.75s/it]


Retrying request 1/3 times...


915it [1:41:38, 48.39s/it]

Streaming output truncated to the last 5000 lines.
An unknown error occurred. Raw response data: {'data': None, 'errors': [{'message': 'Server Error'}], 'extensions': {'is_final': True}}
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
An unknown error occurred. Raw response data: {'data': None, 'errors': [{'message': 'Server Error'}], 'extensions': {'is_final': True}}
An unknown error occurred. Raw response data: {'data': None, 'errors': [{'message': 'Server Error'}], 'extensions': {'is_final': True}}
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
An unknown error occurred. Raw response data: {'data': None, 'errors': [{'message': 'Server Error'}], 'extensions': {'is_

In [15]:
clone_ = df.loc[701:index].copy()
clone_ = clone_.drop(columns = ['prompt'])
clone_['answer'] = list_res
clone_.to_csv(f'/content/drive/MyDrive/largeProject/processData/mrc_6000_{index}.csv', index=False)

In [ ]:
list_res

[(0, 'Theo ngữ cảnh, KPI quý trước là 95,9%.'),
 (1,
  'Tỷ lệ hài lòng của KH dịch vụ chuyển phát chỉ bằng 0,88 lần so với KPI đề ra.'),
 (2, 'Mục tiêu đề ra là 97,2%.')]

In [ ]:
text = "chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?"

print(text, end="", flush=True)

chỉ tiêu Tỷ lệ hài lòng của KH dịch vụ chuyển phát của tổng công ty VTPOST được đánh gía là không đạt chỉ tiêu hiện đã chỉ bằng 0.88 lần so với mục tiêu kpi ban đầu đề ra là 97.2%. Đối với kpi quý trước có được là 95.9%. Giảm 3.8% so với kết quả là với 2/2022. Câu hỏi kpi quý trước là bao nhiêu?